In [1]:
from model import *
from data import *

Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [10]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(20,'data/rocks_unet2/train','image','label',data_gen_args,save_to_dir = "data/rocks_unet2/train/aug",
                        image_color_mode='rgb')
# the batch size should be large enough, otherwise the network fail to learn. 
# if the network is not improving, re-initialize the weights (local-minima)
model = unet(input_size = (256,256,3))
model_checkpoint = ModelCheckpoint('unet_rocks1.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model.load_weights("unet_rocks1.hdf5")
model.fit_generator(myGene,steps_per_epoch=2000,epochs=3,callbacks=[model_checkpoint])

Epoch 1/2
Found 66 images belonging to 1 classes.
Found 66 images belonging to 1 classes.
20/20 [==============================] - 16s 791ms/step - loss: 0.7323 - acc: 0.7886

Epoch 00001: loss improved from inf to 0.73507, saving model to unet_rocks0.hdf5
Epoch 2/2
20/20 [==============================] - 13s 651ms/step - loss: 0.4576 - acc: 0.8197

Epoch 00002: loss improved from 0.73507 to 0.47205, saving model to unet_rocks0.hdf5


### Test your model and save predicted results

In [7]:
testGene = testGenerator("data/rocks_unet2/allrocks", num_image=104, flag_multi_class = True, as_gray=False)
model_p = unet(input_size = (256,256,3))
model_p.load_weights("unet_rocks1.hdf5")
results = model_p.predict_generator(testGene,104,verbose=1)
saveResult("data/rocks_unet2/allrocks",results)

104/104 [==============================] - 73s 703ms/step
